In [7]:
import requests
import psycopg2
from psycopg2 import sql
import os
from dotenv import load_dotenv
import pandas as pd
#from Bitcoin_api import fetch_bitcoin_data
#from new_scrappy import get_bitcoin_news

In [18]:
print("Hello World!")

Hello World!


# BTC

## Connecting to DataBase and Creating Tables

In [4]:
# make sure to load dotenv
from dotenv import load_dotenv
import os
import psycopg2
load_dotenv()

# Database connection paramete(s
Database_URL = os.getenv('DataBase_Url')

#connect to database
conn = psycopg2.connect(Database_URL)
cur = conn.cursor()

#create tables if they don't exist

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_Prices_aws(
    Date DATE PRIMARY KEY,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Volume FLOAT
    
)            
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS Bitcoin_News_aws(
    id SERIAL PRIMARY KEY,
    date DATE,
    title VARCHAR(500)
       
)            
''')

conn.commit()

Python-dotenv could not parse statement starting at line 11


## Insertion of Bitcoin Prices_Update to Table

In [12]:
#function to insert Bitcoin_Prices
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_Prices_Update(Date, Open, High, Low, Close, Volume)
                Values(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (Date) DO NOTHING
                ''',(index, row['1. Open'], row['2. High'], row['3. Low'], row['4. Close'], row['5. Volume']))
        conn.commit()
        cursor.close()
        print("success")
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Insertion of  Bitcoin News_ Update to Table

In [9]:
#function to insert Bitcoin_News
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO Bitcoin_News_Update (id, date, title)
                Values(%s, %s, %s)
                ON CONFLICT (id) DO NOTHING
                ''',(index,row['date'], row['title']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

## Fetching and Inserting Data

In [13]:
#Main Function to orchestrate the data fetching and insertion
def main():
    # fetch Bitcoin Data
    bitcoin_data = fetch_bitcoin_data("BTC","USD")
    #Scrape Bitcoin News data
    bitcoin_titles_df = get_bitcoin_news()
    #connect to postgresSQL database
    conn = psycopg2.connect(Database_URL)

    if conn:
        #Insert data into database
        insert_bitcoin_data(conn,bitcoin_data)
        insert_bitcoin_news(conn,bitcoin_titles_df)

        #close the database connection
        conn.close()
    else:
        print('Failed to connect to the database')

if __name__== '__main__':
    main()

Python-dotenv could not parse statement starting at line 11


success


## dropping

In [3]:
import os
import psycopg2

def drop_tables(table_names, conn):
    """
    Drops the specified tables from the PostgreSQL database.

    Args:
    table_names (list of str): List of table names to be dropped.
    conn: The psycopg2 connection object to the PostgreSQL database.
    """
    # Create a cursor object
    cur = conn.cursor()

    # Iterate over the list of table names and drop each one
    for table_name in table_names:
        drop_table_query = f'DROP TABLE IF EXISTS {table_name} CASCADE'
        cur.execute(drop_table_query)
        print(f'Table {table_name} dropped.')

    # Commit the changes and close the cursor
    conn.commit()
    cur.close()

if __name__ == "__main__":
    # List of table names to be dropped
    table_names = [
        'bitcoin_news_aws'
        
        
        
    ]

    # Connect to the PostgreSQL database using the DATABASE_URL environment variable
    DATABASE_URL = os.getenv('DATABASE_URL')
    conn = psycopg2.connect(DATABASE_URL)

    try:
        # Call the function to drop the tables
        drop_tables(table_names, conn)
    finally:
        if conn:
            conn.close()


Table bitcoin_news_aws dropped.


In [ ]:
import streamlit as st
from psycopg2 import sql
from sqlalchemy import create_engine
import os
import pandas as pd
import requests
from dotenv import load_dotenv
import plotly.express as px

In [1]:
import json

In [22]:
from datetime import datetime

## streamlit_app.py

In [ ]:
import streamlit as st
from psycopg2 import sql
from sqlalchemy import create_engine
import os
import pandas as pd
import requests
from dotenv import load_dotenv


# DataBase connection parameters
load_dotenv()

# 
# Function to connect to the Database
def get_connection():
    # Use st.secrets to get the DATABASE_URL
    DATABASE_URL = st.secrets["DataBase_Url"]
    # Create an engine instance
    engine = create_engine(DATABASE_URL)
    return engine

# Function to fetch bitcoin prices from Database
def fetch_bitcoin_data(engine):
    # Query to fetch data
    query = "SELECT * FROM Bitcoin_Prices_Update ORDER BY date" 

    df = pd.read_sql_query(query, engine)
    
    return df
# Function to fetch bitcoin news from Database
def fetch_bitcoin_news(engine):
    # Query to fetch data
    query = "SELECT * FROM Bitcoin_News_Update" 

    df = pd.read_sql_query(query, engine)
    
    return df

# Get the database connection 
conn= get_connection()
# fetch the bitcoin data and news
bitcoin_prices_df=fetch_bitcoin_data(conn)
bitcoin_news_df=fetch_bitcoin_news(conn)

# Display Bitcoin Prices 
st.title("Bitcoin Data")
st.line_chart(bitcoin_prices_df.set_index("date")["close"])

# Display Bitcoin News 
st.title("Bitcoin News")
st.write(bitcoin_news_df)






In [ ]:
import os
import psycopg2
import json

def save_bitcoin_data(event, context):
    Database_URL = os.getenv('DataBase_Url')
    
    try:
        conn = psycopg2.connect(Database_URL)
        cursor = conn.cursor()
        
        # Parse the incoming event
        data = json.loads(event['body'])
        
        for date, values in data.items():
            cursor.execute('''
                INSERT INTO Bitcoin_Prices_Update(Date, Open, High, Low, Close, Volume)
                Values(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (Date) DO NOTHING
                ''', (date, values['1. open'], values['2. high'], values['3. low'], values['4. close'], values['5. volume']))
        
        conn.commit()
        cursor.close()
        return {
            'statusCode': 200,
            'body': json.dumps('Success')
        }
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error inserting Bitcoin data: {e}')
        }
    finally:
        if conn:
            conn.close()


data={
  "body": "{\"2025-06-04\": {\"1. open\": \"68791.04000000\", \"2. high\": \"68981.88000000\", \"3. low\": \"68640.03000000\", \"4. close\": \"68935.24000000\", \"5. volume\"}}"
}
save_bitcoin_data()